In [1]:
import numpy as np 
import pandas as pd
import random
import os

np.random.seed(45)

In [4]:
no_dup_path='../../Our Labeled Data/No_Dup/no_d/'
contains_dup_path='../../Our Labeled Data/Contains_Dup/contains_d/'

In [5]:
no_dup_files = os.listdir(no_dup_path)
contains_dup_files = os.listdir(contains_dup_path)

### 1. Approach to create labeled data with features - word1, word2, isDuplicate

1. Randomly pick one file from the contains_duplicate dataset
2. Randomly pick two rows
3. Populate the output data with words and isDuplicate value according to the entity number from the two pairs of rows


In this approach we completely ignore the data statistics and context information

In [4]:
df1= pd.DataFrame({'w1':[], 'w2':[], 'count1':[], 'count2':[], 'total1':[], 'total2':[], 'isDuplicate':[]})
for _ in range(100):
    file=random.choice(contains_dup_files)
    file_path=contains_dup_path+file
    tempdf=pd.read_csv(file_path)
    for _ in range(10):
        t=tempdf.sample(n=2).reset_index()
        w1,c1,t1,e1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[0]['Entity_Number']
        w2,c2,t2,e2=t.loc[1]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[1]['Entity_Number']
        isDup=(e1==e2)
        df1= df1.append({'w1':w1, 'w2':w2, 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)

In [5]:
df1[df1['isDuplicate']==0].shape, df1[df1['isDuplicate']==1].shape

((963, 7), (37, 7))

In [6]:
df1

,w1,w2,count1,count2,total1,total2,isDuplicate
0,Euro,euro,12319.0,12319.0,68023.0,68023.0,1.0
1,1999 LUF Euro / Euro,lev,534.0,534.0,68023.0,68023.0,0.0
2,Chilean peso,new sol,204.0,204.0,68023.0,68023.0,0.0
3,1999 ATS euro / euro,Hong Kong dollar,626.0,626.0,68023.0,68023.0,0.0
4,Singapore dollar,US dollar,1944.0,1944.0,68023.0,68023.0,0.0
...,...,...,...,...,...,...,...
995,Japanese Language Proficiency Test,Bellydance Dance Belly Dancing,1.0,1.0,31212.0,31212.0,0.0
996,Dance: African Dance and Drumming,Korean Language & Culture,3.0,3.0,31212.0,31212.0,0.0
997,Michigan Sports,Sport Touring Motorcycles,1.0,1.0,31212.0,31212.0,0.0
998,Paper Crafts,Boomer Wellness and Well-Being,16.0,16.0,31212.0,31212.0,0.0


### 2. Approach to create labeled data with features - word1, word2, isDuplicate

1. Randomly pick one file from the contains_duplicate dataset
2. Randomly pick two rows with a. same entitity number, b. different entitity number
3. Populate the output data with words and isDuplicate value according to the entity number from the two pairs of rows


In this approach we completely ignore the data statistics and context information

In [13]:
df2 = pd.DataFrame({'w1':[], 'w2':[], 'count1':[], 'count2':[], 'total1':[], 'total2':[], 'isDuplicate':[]})

for _ in range(100):
    file=random.choice(contains_dup_files)
    file_path=contains_dup_path+file
    tempdf=pd.read_csv(file_path)
    
    perEntityCatCount=tempdf['Entity_Number'].value_counts()

    for _ in range(3):
        tempdf['entityCount']=tempdf['Entity_Number'].apply(lambda x: perEntityCatCount[x])
        
        try:
            t=tempdf[tempdf['entityCount']>1].sample(n=1).reset_index()
            e=t.loc[0]['Entity_Number']
            t=tempdf[tempdf['Entity_Number']==e].sample(n=2).reset_index()
            w1,c1,t1,e1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[0]['Entity_Number']
            w2,c2,t2,e2=t.loc[1]['Category_Set'], t.loc[1]['Category_Occurrences'], t.loc[1]['Total_Examples'], t.loc[1]['Entity_Number']
            isDup=(e1==e2)
            df2 = df2.append({'w1':w1, 'w2':w2, 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)
        except ValueError:
            pass

        try:
#             t=tempdf[tempdf['entityCount']==1].sample(n=2).reset_index()
            t=tempdf.sample(n=2).reset_index()
            w1,c1,t1,e1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[0]['Entity_Number']
            w2,c2,t2,e2=t.loc[1]['Category_Set'], t.loc[1]['Category_Occurrences'], t.loc[1]['Total_Examples'], t.loc[1]['Entity_Number']
            isDup=(e1==e2)
            df2 = df2.append({'w1':w1, 'w2':w2, 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)

        except ValueError:
            pass

In [14]:
df2['isDuplicate'].value_counts()

df2_nodup = df2.drop_duplicates()

1.0    314
0.0    286
Name: isDuplicate, dtype: int64

# Aproach 3

Get all Pairs of duplicates

In [37]:
df3 = pd.DataFrame({'w1':[], 'w2':[], 'count1':[], 'count2':[], 'total1':[], 'total2':[], 'isDuplicate':[]})

# contains_dup_path

total = 0

for file in contains_dup_files:
    file_path=contains_dup_path+file
    tempdf=pd.read_csv(file_path)
    perEntityCatCount=tempdf['Entity_Number'].value_counts()
    
    tempdf['entityCount']=tempdf['Entity_Number'].apply(lambda x: perEntityCatCount[x])
    
    dupEntityNo = set((tempdf[tempdf['entityCount']>1])['Entity_Number'])
    
    total+=len(dupEntityNo)
    
    for eNo in dupEntityNo:
        t = tempdf[tempdf['Entity_Number']==eNo].reset_index()
        w1,c1,t1,e1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[0]['Entity_Number']
        w2,c2,t2,e2=t.loc[1]['Category_Set'], t.loc[1]['Category_Occurrences'], t.loc[1]['Total_Examples'], t.loc[1]['Entity_Number']
        isDup=(e1==e2)
    
        df3 = df3.append({'w1':w1.strip(), 'w2':w2.strip(), 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)
    
        df3 = df3.append({'w1':w2.strip(), 'w2':w1.strip(), 'count1':c2, 'count2':c1, 'total1':t2, 'total2': t1, 'isDuplicate':isDup}, ignore_index=True)
    
    

In [38]:
df3_nodup = df3.drop_duplicates()
df3_nodup = df3_nodup[df3_nodup['w1']!=df3_nodup['w2']].reset_index()

In [35]:
df3.to_csv('allDuplicates.csv')
df3_nodup.to_csv('allDuplicates_cleanStrings.csv')

In [42]:
df3_nodup.shape

(1782, 8)

In [95]:
df4 = pd.DataFrame({'w1':[], 'w2':[], 'count1':[], 'count2':[], 'total1':[], 'total2':[], 'isDuplicate':[]})

total = 0

for file in no_dup_files:
    file_path=no_dup_path+file
    tempdf=pd.read_csv(file_path)
    
    for _ in range(20):
        if len(tempdf)>2:
            t=tempdf.sample(n=2)
            tempdf = tempdf.drop(t.index)
            t = t.reset_index()
            w1,c1,t1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples']
            w2,c2,t2=t.loc[1]['Category_Set'], t.loc[1]['Category_Occurrences'], t.loc[1]['Total_Examples']
            isDup=0.0
            
            if type(w1)==str and type(w2)==str: 
            
                df4 = df4.append({'w1':str(w1).strip(), 'w2':str(w2).strip(), 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)
                df4 = df4.append({'w1':str(w2).strip(), 'w2':str(w1).strip(), 'count1':c2, 'count2':c1, 'total1':t2, 'total2': t1, 'isDuplicate':isDup}, ignore_index=True)
                
                
for file in contains_dup_files:
    file_path=contains_dup_path+file
    tempdf=pd.read_csv(file_path)
    perEntityCatCount=tempdf['Entity_Number'].value_counts()
    
    tempdf['entityCount']=tempdf['Entity_Number'].apply(lambda x: perEntityCatCount[x])
    
    tempdf = tempdf[tempdf['entityCount']==1]
    
    for _ in range(20):
        if len(tempdf)>2:
            t=tempdf.sample(n=2)
            tempdf = tempdf.drop(t.index)
            t = t.reset_index()
            w1,c1,t1,e1=t.loc[0]['Category_Set'], t.loc[0]['Category_Occurrences'], t.loc[0]['Total_Examples'], t.loc[0]['Entity_Number']
            w2,c2,t2,e2=t.loc[1]['Category_Set'], t.loc[1]['Category_Occurrences'], t.loc[1]['Total_Examples'], t.loc[1]['Entity_Number']
            isDup=(e1==e2)
            
            if type(w1)==str and type(w2)==str: 
                df3 = df3.append({'w1':w1.strip(), 'w2':w2.strip(), 'count1':c1, 'count2':c2, 'total1':t1, 'total2': t2, 'isDuplicate':isDup}, ignore_index=True)
                df3 = df3.append({'w1':w2.strip(), 'w2':w1.strip(), 'count1':c2, 'count2':c1, 'total1':t2, 'total2': t1, 'isDuplicate':isDup}, ignore_index=True)


In [96]:
df4

,w1,w2,count1,count2,total1,total2,isDuplicate
0,Month,Year,39.0,68950.0,374362.0,374362.0,0.0
1,Year,Month,68950.0,39.0,374362.0,374362.0,0.0
2,Hour,Bi-Weekly,1651.0,1.0,374362.0,374362.0,0.0
3,Bi-Weekly,Hour,1.0,1651.0,374362.0,374362.0,0.0
4,AL,NV,7354.0,7779.0,403984.0,403984.0,0.0
...,...,...,...,...,...,...,...
9505,VIEW,LIKE,61086.0,5745.0,72312.0,72312.0,0.0
9506,FOLLOW,COMMENT CREATED,1407.0,1611.0,72312.0,72312.0,0.0
9507,COMMENT CREATED,FOLLOW,1611.0,1407.0,72312.0,72312.0,0.0
9508,A,Y,8892.0,8288.0,374362.0,374362.0,0.0


In [98]:
df4_nodup = df4.drop_duplicates()
df4_nodup = df4_nodup.sample(frac=1).reset_index(drop=True)

In [99]:
df4_nodup.to_csv('NoDuplicates.csv')

In [100]:
df5 = pd.concat((df3_nodup, df4_nodup))

df5 = df5.sample(frac=1).reset_index(drop=True)

df5.to_csv('1AllDuplicates_5NoDuplicates.csv')

In [ ]:
df5['']